# 第9章: ベクトル空間法 (I)

enwiki-20150112-400-r10-105752.txt.bz2は，2015年1月12日時点の英語のWikipedia記事のうち，約400語以上で構成される記事の中から，ランダムに1/10サンプリングした105,752記事のテキストをbzip2形式で圧縮したものである．このテキストをコーパスとして，単語の意味を表すベクトル（分散表現）を学習したい．第9章の前半では，コーパスから作成した単語文脈共起行列に主成分分析を適用し，単語ベクトルを学習する過程を，いくつかの処理に分けて実装する．第9章の後半では，学習で得られた単語ベクトル（300次元）を用い，単語の類似度計算やアナロジー（類推）を行う．

なお，問題83を素直に実装すると，大量（約7GB）の主記憶が必要になる． メモリが不足する場合は，処理を工夫するか，1/100サンプリングのコーパスenwiki-20150112-400-r100-10576.txt.bz2を用いよ．

In [1]:
import numpy as np

# 80. コーパスの整形
文を単語列に変換する最も単純な方法は，空白文字で単語に区切ることである． ただ，この方法では文末のピリオドや括弧などの記号が単語に含まれてしまう． そこで，コーパスの各行のテキストを空白文字でトークンのリストに分割した後，各トークンに以下の処理を施し，単語から記号を除去せよ．

+ トークンの先頭と末尾に出現する次の文字を削除: .,!?;:()[]'"
+ 空文字列となったトークンは削除

以上の処理を適用した後，トークンをスペースで連結してファイルに保存せよ．

In [2]:
# head -n28443 enwiki-20150112-400-r100-10576.txt > enwiki.txt
with open('enwiki.txt', 'r') as f:
    delete_words = ".,!?;:()[]'" + '"'
    corpus = []
    for line in f.readlines():
        for token in line[:-1].split():
            if token[0] in delete_words:
                token = token[1:]
            if len(token) == 0:
                break
            if token[-1] in delete_words:
                token = token[:-1]
            if token:
                corpus.append(token)
with open('corpus.txt', 'w') as f:
    f.write(" ".join(corpus))

# 81. 複合語からなる国名への対処
英語では，複数の語の連接が意味を成すことがある．例えば，アメリカ合衆国は"United States"，イギリスは"United Kingdom"と表現されるが，"United"や"States"，"Kingdom"という単語だけでは，指し示している概念・実体が曖昧である．そこで，コーパス中に含まれる複合語を認識し，複合語を1語として扱うことで，複合語の意味を推定したい．しかしながら，複合語を正確に認定するのは大変むずかしいので，ここでは複合語からなる国名を認定したい．

インターネット上から国名リストを各自で入手し，80のコーパス中に出現する複合語の国名に関して，スペースをアンダーバーに置換せよ．例えば，"United States"は"United_States"，"Isle of Man"は"Isle_of_Man"になるはずである．

In [3]:
# http://www.projectvisa.com/fullcountrylist.asp
with open('country.txt', 'r') as f:
    countries = [x[:-1] for x in f.readlines() if " " in x]

with open('corpus.txt', 'r') as f:
    corpus = f.read()[:-1]

In [4]:
for c in countries:
    corpus = corpus.replace(c, c.replace(" ", "_"))

In [5]:
corpus_list = corpus.split(" ")
dictionary = np.array(list(set(corpus_list)))
T = len(corpus_list)

# 82. 文脈の抽出
81で作成したコーパス中に出現するすべての単語$t$に関して，単語$t$と文脈語$c$のペアをタブ区切り形式ですべて書き出せ．ただし，文脈語の定義は次の通りとする．

+ ある単語$t$の前後$d$単語を文脈語$c$として抽出する（ただし，文脈語に単語$t$そのものは含まない）
+ 単語$t$を選ぶ度に，文脈幅$d$は$\{1,2,3,4,5\}$の範囲でランダムに決める．

In [6]:
def get_d():
    d = np.random.randint(-5, 5)
    if d == 0:
        d = get_d()
    return d

with open("82.txt", "w") as f:
    for i, t in enumerate(corpus_list):
        d = get_d() 
        if (i + d >= 0) and  (i + d < T):
            c = corpus_list[i + d]
            if c != t:
                line = t + "\t" + c + "\n"
                f.write(line)

# 83. 単語／文脈の頻度の計測
82の出力を利用し，以下の出現分布，および定数を求めよ．

+ $f(t,c)$: 単語$t$と文脈語$c$の共起回数
+ $f(t,∗)$: 単語$t$の出現回数
+ $f(∗,c)$: 文脈語$c$の出現回数
+ $N$: 単語と文脈語のペアの総出現回数

In [7]:
import pandas as pd
tc = pd.read_csv("82.txt", sep="\t", header=None, names=["t", "c"])
tc = tc[tc.t != tc.c]

In [8]:
N = len(tc.index)
N

1284196

In [9]:
tc.head()

,t,c
0,Anarchism,political
1,is,a
2,a,advocates
3,political,advocates
4,philosophy,that


In [10]:
f_t = tc.groupby('t').count()
f_c = tc.groupby('c').count()
tc['count'] = tc['t']
f_tc = tc.groupby(list("tc")).count()

In [11]:
f_t.ix['a']

c    24810
Name: a, dtype: int64

In [12]:
f_c.ix['a']

t    24842
Name: a, dtype: int64

In [13]:
f_tc.ix[('a', 'that')][0]

191

# 84. 単語文脈行列の作成
83の出力を利用し，単語文脈行列$X$を作成せよ．ただし，行列$X$の各要素$X_{tc}$は次のように定義する．

+ $f(t,c)≥10$ならば，$X_{tc}=PPMI(t,c)=max\{log \frac{N×f(t,c)}{f(t,∗)×f(∗,c)},0\}$
+ $f(t,c)<10$ならば，$X_{tc}=0$

ここで，$PPMI(t,c)$はPositive Pointwise Mutual Information（正の相互情報量）と呼ばれる統計量である．
なお，行列$X$の行数・列数は数百万オーダとなり，行列のすべての要素を主記憶上に載せることは無理なので注意すること．幸い，行列$X$のほとんどの要素は$0$になるので，非$0$の要素だけを書き出せばよい．

In [14]:
f_tc[f_tc['count'] >= 10].head()

count
t  c         
1  of      13
   on      11
   the     19
   was     10
10 in      10

In [15]:
len(f_tc[f_tc['count'] >= 10])

8520

In [16]:
from scipy import sparse
from math import log
W = len(dictionary)
X = sparse.lil_matrix((W, W))

In [17]:
dictionary[100]

'vile'

In [20]:
np.where(dictionary == 'is')[0][0]

26197

In [21]:
dictionary[26197]

'is'

In [22]:
X_df = f_tc[f_tc['count'] >= 10].reset_index()
X_df.head()

,t,c,count
0,1,of,13
1,1,on,11
2,1,the,19
3,1,was,10
4,10,in,10


In [23]:
f_t = f_t.reset_index()
f_t.columns = [['t', 'ft']]

In [24]:
f_c = f_c.reset_index()
f_c.columns = [['c', 'fc']]

In [25]:
X_df = pd.merge(X_df, f_t, on='t')
X_df = pd.merge(X_df, f_c, on='c')
X_df.head()

,t,c,count,ft,fc
0,1,of,13,441,49921
1,10,of,15,276,49921
2,100,of,11,647,49921
3,18,of,153,1520,49921
4,2000,of,58,621,49921


In [26]:
X_df['ppmi1'] = (N * X_df['count']) / (X_df['ft'] * X_df['fc'])

In [27]:
X_df.head()

,t,c,count,ft,fc,ppmi1
0,1,of,13,441,49921,0.758321
1,10,of,15,276,49921,1.398074
2,100,of,11,647,49921,0.437357
3,18,of,153,1520,49921,2.589381
4,2000,of,58,621,49921,2.402616


In [28]:
len(X_df[X_df['ppmi1'] > 1])

6409

In [29]:
with open('X.txt', 'w') as f:
    line = "t_i,c_i,ppmi\n"
    f.write(line)
    for row in X_df[X_df['ppmi1'] > 1].iterrows():
        t = row[1]['t']
        c = row[1]['c']
        t_i = np.where(dictionary == t)[0][0]
        c_i = np.where(dictionary == c)[0][0]
        ppmi = log(row[1]['ppmi1'])
        line = str(t_i) + "," + str(c_i) + "," + str(ppmi) + "\n"
        f.write(line)

In [30]:
X_csv = pd.read_csv('X.txt')
for row in X_csv.iterrows():
    X[row[1]['t_i'], row[1]['c_i']] = row[1]['ppmi']

# 85. 主成分分析による次元圧縮
84で得られた単語文脈行列に対して，主成分分析を適用し，単語の意味ベクトルを300次元に圧縮せよ．

In [31]:
# https://stackoverflow.com/questions/33603787/performing-pca-on-large-sparse-matrix-by-using-sklearn
from sklearn.decomposition import TruncatedSVD
clf = TruncatedSVD(300)
Xpca = clf.fit_transform(X)

In [32]:
Xpca

array([[ -4.68889740e-15,   3.94710245e-15,  -6.07140799e-15, ...,
          1.92331748e-16,   1.67572173e-16,  -1.54882369e-16],
       [ -4.71660897e-16,   5.45429751e-15,  -1.60533511e-15, ...,
         -1.36382128e-16,  -1.94146470e-16,  -1.12592492e-16],
       [ -3.22796058e-16,   8.46850367e-16,  -1.00355398e-15, ...,
          1.20282575e-16,  -9.80202510e-17,  -2.71811618e-16],
       ..., 
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          0.00000000e+00,  -0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          0.00000000e+00,  -0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          0.00000000e+00,  -0.00000000e+00,   0.00000000e+00]])

# 86. 単語ベクトルの表示
85で得た単語の意味ベクトルを読み込み，"United States"のベクトルを表示せよ．ただし，"United States"は内部的には"United_States"と表現されていることに注意せよ．

In [33]:
def vec(word):
    i = np.where(dictionary == word)[0][0]
    return Xpca[i].reshape(1, -1)

In [34]:
vec("United_States")

array([[  7.58660899e-01,  -5.08962569e-01,   1.32330366e+00,
          1.54863439e-01,  -1.26987219e-01,   2.19281149e-01,
          3.59846323e+00,   4.35852928e-01,  -1.59769966e+00,
         -2.21568188e-01,   1.07949071e+00,   1.14404089e+00,
          9.25374939e-01,   3.59771463e-01,   3.90410694e+00,
          5.55161332e-01,  -1.70299072e+00,  -1.02625177e+00,
         -1.61096673e+00,  -8.65139448e-01,   1.20757269e-01,
         -7.27082088e-01,  -1.11683726e+00,  -4.33329563e-01,
          5.23616016e-01,  -3.78601913e-01,  -2.99316636e-02,
          4.50883910e-01,  -3.09245732e-01,  -6.06753826e-01,
          5.58668317e-02,  -1.83836602e-01,  -6.51845067e-01,
          4.80972015e-01,   3.20178893e-01,  -4.62340266e-01,
         -2.97092514e-01,  -1.50879887e-01,   2.63497405e-01,
         -7.27013907e-02,   4.95360099e-02,   1.62227699e-01,
          8.06535592e-01,  -3.78516184e-01,   1.73879008e-01,
          6.58097987e-01,   1.17975763e+00,  -4.26480253e-03,
        

# 87. 単語の類似度
85で得た単語の意味ベクトルを読み込み，"United States"と"U.S."のコサイン類似度を計算せよ．ただし，"U.S."は内部的に"U.S"と表現されていることに注意せよ．

In [35]:
from sklearn.metrics.pairwise import cosine_similarity
def get_word_cs(word1, word2):
    return cosine_similarity(vec(word1), vec(word2))[0][0]
get_word_cs("United_States", "U.S")

0.058618268924742457

# 88. 類似度の高い単語10件
85で得た単語の意味ベクトルを読み込み，"England"とコサイン類似度が高い10語と，その類似度を出力せよ．

In [38]:
def get_top10(word):
    lst=[]
    for i, w in enumerate(Xpca[:1000]):
        cs = cosine_similarity(vec(word), w.reshape(1, -1))[0][0]
        lst.append((dictionary[i], cs))
    return pd.DataFrame(lst)

In [39]:
eng_top10 = get_top10("England")

In [45]:
eng_top10.sort_values(1, ascending=False).head(10)

,0,1
617,May,0.923128
927,Europe,0.843394
299,music,0.813579
961,election,0.781285
420,then,0.282207
725,influence,0.203258
110,manibus,0.171149
217,Ben-Gurion,0.156187
7,Shinde,0.155307
206,skyscrapers,0.141807


# 89. 加法構成性によるアナロジー
85で得た単語の意味ベクトルを読み込み，vec("Spain") - vec("Madrid") + vec("Athens")を計算し，そのベクトルと類似度の高い10語とその類似度を出力せよ

In [46]:
vec89 = vec("Spain") - vec("Madrid") + vec("Athens")
lst=[]
for i, w in enumerate(Xpca[:1000]):
    cs = cosine_similarity(vec89, w.reshape(1, -1))
    lst.append((dictionary[i], cs[0][0]))
df89 = pd.DataFrame(lst)

In [47]:
df89.sort_values(1, ascending=False).head(10)

,0,1
671,outside,1.000000
8,larger,1.000000
246,road,1.000000
531,Athens,1.000000
980,No,1.000000
961,election,0.499605
617,May,0.183763
10,Vespucius),0.113157
211,Derzhanski,0.092653
7,Shinde,0.091238
